### OLTP Database Schema

The OLTP database is a PostgreSQL database that contains the data of the student notes with 4 tables:

#### Table: `student`
| Column                | Data Type     | Description                                   |
|-----------------------|---------------|-----------------------------------------------|
| `id`                 | SERIAL        | Unique identifier for each student.          |
| `apo`                | VARCHAR(50)   | Administrative ID.                           |
| `nom`                | VARCHAR(50)   | Last name of the student.                    |
| `prenom`             | VARCHAR(50)   | First name of the student.                   |
| `sexe`               | CHAR(1)       | Gender (M/F).                                |
| `birthdate`          | DATE          | Birth date of the student.                   |
| `number_of_ajournements` | INTEGER     | Number of failed attempts.                   |
| `promo`              | VARCHAR(50)   | Promotion/Year group.                        |

#### Table: `modules`
| Column     | Data Type     | Description                                   |
|------------|---------------|-----------------------------------------------|
| `id`      | SERIAL        | Unique identifier for each module.            |
| `code`    | VARCHAR(50)   | Module code.                                  |
| `name`    | VARCHAR(100)  | Module name.                                  |
| `filiere` | VARCHAR(100)  | Field of study.                               |
| `coeff`   | NUMERIC(5,2)  | Coefficient (weight of the module).           |
| `profid`  | INTEGER       | FK to professor table.                        |
| `semester`| VARCHAR(50)   | Semester in which the module is offered.      |

#### Table: `prof`
| Column       | Data Type     | Description                                   |
|--------------|---------------|-----------------------------------------------|
| `id`        | SERIAL        | Unique identifier for each professor.         |
| `name`      | VARCHAR(100)  | Full name of the professor.                   |
| `departement`| VARCHAR(100)  | Department the professor belongs to.          |
| `email`     | VARCHAR(100)  | Email address of the professor.               |

#### Table: `notes`
| Column       | Data Type     | Description                                   |
|--------------|---------------|-----------------------------------------------|
| `id`        | SERIAL        | Unique identifier for each note record.       |
| `studentid` | INTEGER       | FK to student table.                          |
| `moduleid`  | INTEGER       | FK to module table.                           |
| `note`      | NUMERIC(5,2)  | Grade obtained in the module.                 |
| `rattrapage`| BOOLEAN       | Indicates if the note is remedial (1 = Yes).  |

---

### Data Warehouse Schema

The Data Warehouse is a PostgreSQL database that contains the data of the student notes with 4 tables:

#### Facts Table: `Fact_Student_Performance`
| Column       | Data Type      | Description                                   |
|--------------|----------------|-----------------------------------------------|
| `fact_id`   | SERIAL         | Unique identifier for each fact record.       |
| `student_id`| INTEGER        | FK to `Dim_Student`.                          |
| `module_id` | INTEGER        | FK to `Dim_Module`.                           |
| `time_id`   | INTEGER        | FK to `Dim_Time`.                             |
| `prof_id`   | INTEGER        | FK to `Dim_Professor`.                        |
| `note`      | NUMERIC(5,2)   | Grade obtained in the module.                 |
| `rattrapage`| BOOLEAN        | Indicates if the note is remedial (1 = Yes).  |
| `absences`  | INTEGER        | Number of absences for the module.            |

#### Dimension Table: `Dim_Student`
| Column                     | Data Type     | Description                                   |
|----------------------------|---------------|-----------------------------------------------|
| `student_id`              | SERIAL        | Unique identifier for each student.          |
| `apo`                     | VARCHAR(50)   | Administrative ID.                           |
| `name`                    | VARCHAR(100)  | Full name (concatenation of nom and prénom). |
| `sexe`                    | CHAR(1)       | Gender (M/F).                                |
| `birthdate`               | DATE          | Birth date of the student.                   |
| `promo`                   | VARCHAR(50)   | Promotion/Year group.                        |
| `number_of_ajournements`  | INTEGER       | Number of failed attempts.                   |

#### Dimension Table: `Dim_Module`
| Column       | Data Type      | Description                                   |
|--------------|----------------|-----------------------------------------------|
| `module_id` | SERIAL         | Unique identifier for each module.            |
| `code`      | VARCHAR(50)    | Module code.                                  |
| `name`      | VARCHAR(100)   | Module name.                                  |
| `filiere`   | VARCHAR(100)   | Field of study.                               |
| `coeff`     | NUMERIC(5,2)   | Coefficient (weight of the module).           |
| `semester`  | VARCHAR(50)    | Semester in which the module is offered.      |

#### Dimension Table: `Dim_Professor`
| Column       | Data Type      | Description                                   |
|--------------|----------------|-----------------------------------------------|
| `prof_id`   | SERIAL         | Unique identifier for each professor.         |
| `name`      | VARCHAR(100)   | Full name of the professor.                   |
| `department`| VARCHAR(100)   | Department the professor belongs to.          |
| `email`     | VARCHAR(100)   | Email address of the professor.               |

#### Dimension Table: `Dim_Time`
| Column       | Data Type      | Description                                   |
|--------------|----------------|-----------------------------------------------|
| `time_id`   | SERIAL         | Unique identifier for each time record.       |
| `semester`  | VARCHAR(50)    | The semester (e.g., "Spring", "Fall").        |
| `academic_year` | VARCHAR(9) | The academic year (e.g., "2023–2024").        |


In [2]:
import pandas as pd 
import psycopg2
import os



In [3]:
#print the current working directory
base_dir = os.getcwd()
base_dir=base_dir+'/bi'
print("Base directory:", base_dir)

promo_folders = [f for f in os.listdir(base_dir) if f.startswith('promo_')]
print("Promo folders found:", promo_folders)


Base directory: C:\Users\ezzou\OneDrive\Desktop\Bi data/bi
Promo folders found: ['promo_2019', 'promo_2020', 'promo_2021', 'promo_2022', 'promo_2023', 'promo_2024']


In [4]:
import os
import pandas as pd

# Base directory
base_dir = r'C:\Users\ezzou\OneDrive\Desktop\Bi data\bi'

# List of promo folders
promo_folders = [f for f in os.listdir(base_dir) if f.startswith('promo_')]
print("Promo folders found:", promo_folders)

# Dictionary to store DataFrames for each module within each promo
promo_module_dfs = {}

# Iterate through each promo folder
for promo in promo_folders:
    promo_path = os.path.join(base_dir, promo, 'absence_files')
    if os.path.exists(promo_path):
        print("Processing promo folder:", promo)
        # Dictionary to store DataFrames for each module in the current promo
        module_dfs = {}
        # Iterate through each CSV file in the absences files folder
        for file in os.listdir(promo_path):
            if file.endswith('.csv'):
                file_path = os.path.join(promo_path, file)
                print("Processing file:", file_path)
                # Read the CSV file
                df = pd.read_csv(file_path)
                print(df)
                
                # Sum the absences for each student
                df['total_absences'] = df.iloc[:, 3:].sum(axis=1)
                print(df)
                
                # Store the DataFrame in the module_dfs dictionary
                module_name = os.path.splitext(file)[0]
                module_dfs[module_name] = df[['nom', 'prenom', 'apogee', 'total_absences']]
        
        # Store the module DataFrames in the promo_module_dfs dictionary
        promo_module_dfs[promo] = module_dfs
    else:
        print(f"Directory does not exist: {promo_path}")

# Example: Accessing a DataFrame for a specific module in a specific promo
promo_name = 'promo_2019'
module_name = 'absence_Administration et Optimisation des BD'
df = promo_module_dfs[promo_name][module_name]
print(df)

Promo folders found: ['promo_2019', 'promo_2020', 'promo_2021', 'promo_2022', 'promo_2023', 'promo_2024']
Processing promo folder: promo_2019
Processing file: C:\Users\ezzou\OneDrive\Desktop\Bi data\bi\promo_2019\absence_files\absence_Administration et Optimisation des BD.csv
          nom   prenom    apogee  cour1  cour2  cour3  cour4  cour5  cour6  \
0   Benkirane    Fouad  66141774      1      1      0      0      1      0   
1       Zayed   Dounia  34462736      0      1      1      0      0      1   
2      El-Bay   Dounia  87205287      1      0      1      1      1      1   
3   Cherkaoui    Hajar  61461023      1      1      1      1      1      1   
4     Kabbour     Dari  92839140      1      1      1      1      0      1   
..        ...      ...       ...    ...    ...    ...    ...    ...    ...   
65      Mejri  Youssef  30141586      1      1      1      1      0      0   
66  Benkirane   Loubna  61072533      1      1      1      1      1      1   
67  El-Qasimi     Zin

In [5]:
promo_name = 'promo_2019'
module_name = 'absence_Administration et Optimisation des BD'
df = promo_module_dfs[promo_name][module_name]
print(df)

          nom   prenom    apogee  total_absences
0   Benkirane    Fouad  66141774              26
1       Zayed   Dounia  34462736              25
2      El-Bay   Dounia  87205287              28
3   Cherkaoui    Hajar  61461023              31
4     Kabbour     Dari  92839140              31
..        ...      ...       ...             ...
65      Mejri  Youssef  30141586              25
66  Benkirane   Loubna  61072533              30
67  El-Qasimi     Zine  17583209              30
68     Chafik   Hanane  74588410              32
69   El-Basri    Salim  69535522              23

[70 rows x 4 columns]


In [6]:
prof_df=pd.read_csv('./bi/prof.csv')
prof_df.head()

,id,name,departement,email
0,1,Anouar ABTOY,GI,anouar@uae.ac.ma
1,2,Mohammed AL ACHHAB,GI,mohammed@uae.ac.ma
2,3,Zineb BESRI,GI,zineb@uae.ac.ma
3,4,Mohamed Yassin CHKOURI,GI,mohamedyassin@uae.ac.ma
4,5,Mohamed CHRAYAH,GI,mohamed@uae.ac.ma


In [7]:
module_df=pd.read_csv('./bi/modules.csv')
module_df.head()


,id,code,name,filiere,coeff,profID,semester,absence_bias,median_mark
0,1,TC1,Langues et Communication I,GI,0.5,17,S1,2,14.0
1,2,TC1,Langues et Communication I,GI,0.5,19,S1,2,14.0
2,3,TC2,Management I,GI,1.0,21,S1,9,13.5
3,4,TC3,Probabilités Statistiques et Calcul Stochastique,GI,1.0,22,S1,1,14.0
4,5,TC4,Réseaux Informatiques I,GI,1.0,8,S1,5,12.5


In [8]:
#print the columns of the module_df and prof_df
print("Columns of the module_df:")
print(module_df.columns)
print("\nColumns of the prof_df:")
print(prof_df.columns)


Columns of the module_df:
Index(['id', 'code', 'name', 'filiere', 'coeff', 'profID', 'semester',
       'absence_bias', 'median_mark'],
      dtype='object')

Columns of the prof_df:
Index(['id', 'name', 'departement', 'email'], dtype='object')


In [9]:
# Merge module_df and prof_df on the profID and id columns
merged_df = pd.merge(module_df, prof_df, left_on='profID', right_on='id', suffixes=('_module', '_prof'))

# Drop the redundant 'id_prof' column after the merge
merged_df.drop(columns=['id_prof'], inplace=True)
merged_df.drop(columns=['absence_bias','median_mark'], inplace=True)

# Print the merged DataFrame
merged_df.head()

,id_module,code,name_module,filiere,coeff,profID,semester,name_prof,departement,email
0,1,TC1,Langues et Communication I,GI,0.5,17,S1,Belmir Ibrahim,DHM,belmir@uae.ac.ma
1,2,TC1,Langues et Communication I,GI,0.5,19,S1,MALLA HUSSEIN Sawsan,DHM,malla@uae.ac.ma
2,3,TC2,Management I,GI,1.0,21,S1,Mohammed Rida Charat,SMAD,mohammedrida@uae.ac.ma
3,4,TC3,Probabilités Statistiques et Calcul Stochastique,GI,1.0,22,S1,EL BIARI Aouatef,SMAD,aouatef@uae.ac.ma
4,5,TC4,Réseaux Informatiques I,GI,1.0,8,S1,Yasser EL KHAMLICHI,GI,yasser@uae.ac.ma


In [10]:
#Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI",
    user="postgres",
    password="root"
)
#Create a cursor object using the cursor() method
cursor = conn.cursor()
cursor.execute("SELECT * FROM students")
students = cursor.fetchall()
#drop a column the third column
students = [student[:2] + student[3:] for student in students]
#transform the students data into a pandas DataFrame 
students_df = pd.DataFrame(students, columns=['id', 'apo', 'nom', 'prenom', 'sexe', 'birthdate', 'number_of_ajournements', 'promo'])
students_df.head()


,id,apo,nom,prenom,sexe,birthdate,number_of_ajournements,promo
0,1,66141774,Benkirane,Fouad,M,2000-06-25,1,2019
1,2,34462736,Zayed,Dounia,F,2000-02-13,0,2019
2,3,87205287,El-Bay,Dounia,F,2000-08-31,0,2019
3,4,61461023,Cherkaoui,Hajar,F,2000-09-10,0,2019
4,5,92839140,Kabbour,Dari,M,2000-11-05,0,2019


In [11]:
#get the marks data from the database
cursor.execute("SELECT * FROM marks")
marks = cursor.fetchall()
#get the columns names
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'marks'")
columns = cursor.fetchall()
columns = [column[0] for column in columns]
#transform the marks data into a pandas DataFrame
marks_df = pd.DataFrame(marks, columns=columns)
marks_df.head()


,student_apogee,module_id,mark,is_split2,mark_submodule1,mark_submodule2,year
0,66141774,8,15.46,True,14.73,16.20,2016/2017
1,34462736,8,12.75,True,12.00,13.49,2016/2017
2,87205287,8,17.09,True,20.00,14.19,2016/2017
3,61461023,8,15.40,True,17.72,13.09,2016/2017
4,92839140,8,16.07,True,19.50,12.64,2016/2017


In [12]:
#get all the keys on the promo_module_dfs dictionary 
promo_module_keys=promo_module_dfs.keys()
print(promo_module_keys)


dict_keys(['promo_2019', 'promo_2020', 'promo_2021', 'promo_2022', 'promo_2023', 'promo_2024'])


In [13]:
#for absence_Langues et Communication I1 and absence_Langues et Communication I2 in each promo create a DataFrame with the columns: 'nom', 'prenom', 'apogee', 'total_absences' named absence_Langues et Communication I and the total_absences column should be the sum of the total_absences of absence_Langues et Communication I1 and absence_Langues et Communication I2 and add it to the promo_module_dfs dictionary delete the absence_Langues et Communication I1 and absence_Langues et Communication I2 DataFrames from the promo_module_dfs dictionary
for promo in promo_module_keys:
  module_keys = promo_module_dfs[promo].keys()
  absence_Langues_et_Communication_I1 = promo_module_dfs[promo]['absence_Langues et Communication I1']
  absence_Langues_et_Communication_I2 = promo_module_dfs[promo]['absence_Langues et Communication I2']
  absence_Langues_et_Communication_I1['total_absences'] = absence_Langues_et_Communication_I1.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I2['total_absences'] = absence_Langues_et_Communication_I2.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I = pd.concat([absence_Langues_et_Communication_I1, absence_Langues_et_Communication_I2])
  absence_Langues_et_Communication_I.drop(columns=['total_absences'], inplace=True)
  promo_module_dfs[promo]['absence_Langues et Communication I'] = absence_Langues_et_Communication_I
  del promo_module_dfs[promo]['absence_Langues et Communication I1']
  del promo_module_dfs[promo]['absence_Langues et Communication I2']

In [14]:
#make one absent df for absence_Modélisation et Programmation Objet
for promo in promo_module_keys:
  module_keys = promo_module_dfs[promo].keys()
  absence_Langues_et_Communication_I1 = promo_module_dfs[promo]['absence_Modélisation et Programmation Objet1']
  absence_Langues_et_Communication_I2 = promo_module_dfs[promo]['absence_Modélisation et Programmation Objet2']
  absence_Langues_et_Communication_I1['total_absences'] = absence_Langues_et_Communication_I1.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I2['total_absences'] = absence_Langues_et_Communication_I2.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I['total_absences'] = absence_Langues_et_Communication_I1['total_absences'] + absence_Langues_et_Communication_I2['total_absences']
  promo_module_dfs[promo]['absence_Modélisation et Programmation Objet'] = absence_Langues_et_Communication_I
  del promo_module_dfs[promo]['absence_Modélisation et Programmation Objet1']
  del promo_module_dfs[promo]['absence_Modélisation et Programmation Objet2']

In [15]:
#make one absent df for 
for promo in promo_module_keys:
  module_keys = promo_module_dfs[promo].keys()
  absence_Langues_et_Communication_I1 = promo_module_dfs[promo]['absence_Technologies DotNet et JEE1']
  absence_Langues_et_Communication_I2 = promo_module_dfs[promo]['absence_Technologies DotNet et JEE2']
  absence_Langues_et_Communication_I1['total_absences'] = absence_Langues_et_Communication_I1.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I2['total_absences'] = absence_Langues_et_Communication_I2.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I ['total_absences'] = absence_Langues_et_Communication_I1['total_absences'] + absence_Langues_et_Communication_I2['total_absences']
  promo_module_dfs[promo]['absence_Technologies DotNet et JEE'] = absence_Langues_et_Communication_I
  del promo_module_dfs[promo]['absence_Technologies DotNet et JEE1']
  del promo_module_dfs[promo]['absence_Technologies DotNet et JEE2']

In [16]:
#do the same for absence_Système d’Intégration et Progiciel2 and absence_Système d’Intégration et Progiciel1
for promo in promo_module_keys:
  print(f"Promo: {promo}")
  module_keys = promo_module_dfs[promo].keys()
  absence_Langues_et_Communication_I1 = promo_module_dfs[promo]['absence_Système d’Intégration et Progiciel1']
  absence_Langues_et_Communication_I2 = promo_module_dfs[promo]['absence_Système d’Intégration et Progiciel2']
  absence_Langues_et_Communication_I1['total_absences'] = absence_Langues_et_Communication_I1.iloc[:, 3:].sum(axis=1)
  absence_Langues_et_Communication_I2['total_absences'] = absence_Langues_et_Communication_I2.iloc[:, 3:].sum(axis=1)
  #sum the total_absences of absence_Système d’Intégration et Progiciel1 and absence_Système d’Intégration et Progiciel2
  absence_Langues_et_Communication_I ['total_absences'] = absence_Langues_et_Communication_I1['total_absences'] + absence_Langues_et_Communication_I2['total_absences']
  promo_module_dfs[promo]['absence_Système d’Intégration et Progiciel'] = absence_Langues_et_Communication_I
  del promo_module_dfs[promo]['absence_Système d’Intégration et Progiciel1']
  del promo_module_dfs[promo]['absence_Système d’Intégration et Progiciel2']


Promo: promo_2019
Promo: promo_2020
Promo: promo_2021
Promo: promo_2022
Promo: promo_2023
Promo: promo_2024


In [17]:
promo_module_dfs['promo_2019']['absence_Système d’Intégration et Progiciel']
#null values in the absence_Système d’Intégration et Progiciel DataFrame
promo_module_dfs['promo_2019']['absence_Système d’Intégration et Progiciel'].isnull().sum()

nom               0
prenom            0
apogee            0
total_absences    0
dtype: int64

In [18]:
#now i want to create a result DataFrame for each promo that contains the columns: 'nom', 'prenom', 'apogee', 'total_absences' 'promo' and 'module' 


In [19]:
result_dfs = []

for promo in promo_module_dfs.keys():
    for module in promo_module_dfs[promo].keys():
        df = promo_module_dfs[promo][module].copy()
        df['promo'] = promo
        df['module'] = module
        result_dfs.append(df)

# Concatenate all the DataFrames into a single DataFrame
final_result_df = pd.concat(result_dfs, ignore_index=True)

In [20]:
final_result_df.head()

,nom,prenom,apogee,total_absences,promo,module
0,Benkirane,Fouad,66141774,26.0,promo_2019,absence_Administration et Optimisation des BD
1,Zayed,Dounia,34462736,25.0,promo_2019,absence_Administration et Optimisation des BD
2,El-Bay,Dounia,87205287,28.0,promo_2019,absence_Administration et Optimisation des BD
3,Cherkaoui,Hajar,61461023,31.0,promo_2019,absence_Administration et Optimisation des BD
4,Kabbour,Dari,92839140,31.0,promo_2019,absence_Administration et Optimisation des BD


In [21]:
final_result_df.sample(10)

,nom,prenom,apogee,total_absences,promo,module
14421,Ibrahim,Sara,76734250,31.0,promo_2024,absence_Technologies DotNet et JEE
12498,Amrani,Amina,89538670,10.0,promo_2024,absence_Business Intelligence
11831,Ibrahim,Sara,76734250,31.0,promo_2023,absence_Modélisation et Programmation Objet
4358,Zarrouk,Adil,88904661,NaN,promo_2020,absence_Langues et Communication I
3742,Maki,Mouna,20470749,23.0,promo_2020,absence_Réseaux informatique II
3604,Tazi,Hanane,34194426,8.0,promo_2020,absence_Programmation Web I
12530,Madani,Jana,56543637,18.0,promo_2024,absence_Intelligence Artificielle
10930,Ait-Sidi,Dounia,85974855,26.0,promo_2023,absence_Programmation Web I
3970,Essamadi,Mouna,45029871,22.0,promo_2020,absence_Systèmes d’Information et Bases de Don...
1248,Nadi,Hajar,18388787,27.0,promo_2019,absence_Programmation web II


In [22]:
#in the module column replace delte absence_
final_result_df['module'] = final_result_df['module'].str.replace('absence_', '')
final_result_df.head()

,nom,prenom,apogee,total_absences,promo,module
0,Benkirane,Fouad,66141774,26.0,promo_2019,Administration et Optimisation des BD
1,Zayed,Dounia,34462736,25.0,promo_2019,Administration et Optimisation des BD
2,El-Bay,Dounia,87205287,28.0,promo_2019,Administration et Optimisation des BD
3,Cherkaoui,Hajar,61461023,31.0,promo_2019,Administration et Optimisation des BD
4,Kabbour,Dari,92839140,31.0,promo_2019,Administration et Optimisation des BD


In [23]:
cursor.execute("SELECT * FROM modules")
modules = cursor.fetchall()
columns = [column[0] for column in cursor.description]
modules_df = pd.DataFrame(modules, columns=columns)

In [24]:
#the number of nan values in the final_result_df
final_result_df.isnull().sum()


nom                 0
prenom              0
apogee              0
total_absences    700
promo               0
module              0
dtype: int64

In [25]:
#sahpe (number of rows and columns) of the final_result_df
final_result_df.shape


(14700, 6)

In [26]:
final_result_df.head()


,nom,prenom,apogee,total_absences,promo,module
0,Benkirane,Fouad,66141774,26.0,promo_2019,Administration et Optimisation des BD
1,Zayed,Dounia,34462736,25.0,promo_2019,Administration et Optimisation des BD
2,El-Bay,Dounia,87205287,28.0,promo_2019,Administration et Optimisation des BD
3,Cherkaoui,Hajar,61461023,31.0,promo_2019,Administration et Optimisation des BD
4,Kabbour,Dari,92839140,31.0,promo_2019,Administration et Optimisation des BD


In [27]:
final_result_df.dropna(inplace=True)


In [28]:
final_result_df.shape

(14000, 6)

In [29]:
final_result_df=final_result_df.drop(columns=['nom','prenom'])

In [30]:
final_result_df.head()

,apogee,total_absences,promo,module
0,66141774,26.0,promo_2019,Administration et Optimisation des BD
1,34462736,25.0,promo_2019,Administration et Optimisation des BD
2,87205287,28.0,promo_2019,Administration et Optimisation des BD
3,61461023,31.0,promo_2019,Administration et Optimisation des BD
4,92839140,31.0,promo_2019,Administration et Optimisation des BD


In [31]:
#get the modules data from the database
cursor.execute("SELECT * FROM modules")
modules = cursor.fetchall()
columns = [column[0] for column in cursor.description]
modules_df = pd.DataFrame(modules, columns=columns)
modules_df.head()

,id,code,name,filiere,coeff,prof_id,semester
0,1,TC1,Langues et Communication I,GI,0.5,17,S1
1,2,TC1,Langues et Communication I,GI,0.5,19,S1
2,3,TC2,Management I,GI,1.0,21,S1
3,4,TC3,Probabilités Statistiques et Calcul Stochastique,GI,1.0,22,S1
4,5,TC4,Réseaux Informatiques I,GI,1.0,8,S1


In [32]:
#get the unique values in the module column of the final_result_df
final_result_df['module'].unique()


array(['Administration et Optimisation des BD',
       'Architecture des Ordinateurs et Assembleur',
       'Base de Données Relationnelle-Objet et Répartie',
       'Business Intelligence', 'Intelligence Artificielle',
       'Java Avancé', 'Langues et Communication II',
       'Langues et Communication III', 'Management I', 'Management II',
       'Management III', 'Méthodologies et Génie Logiciel',
       'Probabilités Statistiques et Calcul Stochastique',
       'Programmation C Avancé et Complexité',
       'Programmation des Systèmes',
       'Programmation Réseaux et Sécurité Informatique',
       'Programmation Web I', 'Programmation web II',
       'Réseaux informatique II', 'Réseaux Informatiques I',
       'Systèmes d’exploitation',
       'Systèmes d’Information et Bases de Données Relationnelles',
       'Théories des Graphes et Recherche Opérationnelle',
       'Théories des Langages et Compilation',
       'Urbanisation des Systèmes d’Information', 'Vision Artificielle',

In [33]:
# Check if all the modules in the final_result_df are in the modules_df
modules_in_final_result = final_result_df['module'].unique()
modules_in_modules_df = modules_df['name'].unique()

# Find modules that are not in the modules_df
missing_modules = [module for module in modules_in_final_result if module not in modules_in_modules_df]

# Print the missing modules
print("Modules not found in modules_df:", missing_modules)

Modules not found in modules_df: []


In [34]:
# replace the module name with it id in the final_result_df
final_result_df = final_result_df.merge(modules_df, left_on='module', right_on='name', suffixes=('_absence', '_module'))
final_result_df.head()

,apogee,total_absences,promo,module,id,code,name,filiere,coeff,prof_id,semester
0,66141774,26.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
1,34462736,25.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
2,87205287,28.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
3,61461023,31.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
4,92839140,31.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4


In [35]:
#drop the name column
final_result_df.head()

,apogee,total_absences,promo,module,id,code,name,filiere,coeff,prof_id,semester
0,66141774,26.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
1,34462736,25.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
2,87205287,28.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
3,61461023,31.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4
4,92839140,31.0,promo_2019,Administration et Optimisation des BD,24,GINF42,Administration et Optimisation des BD,GI,1.0,12,S4


In [36]:
#get the students data from the database
cursor.execute("SELECT * FROM students")
students = cursor.fetchall()
columns = [column[0] for column in cursor.description]
students_df = pd.DataFrame(students, columns=columns)
students_df.head()

,id,apogee,code_apo,nom,prenom,sexe,birthdate,number_of_ajournements,promo_year
0,1,66141774,66141774,Benkirane,Fouad,M,2000-06-25,1,2019
1,2,34462736,34462736,Zayed,Dounia,F,2000-02-13,0,2019
2,3,87205287,87205287,El-Bay,Dounia,F,2000-08-31,0,2019
3,4,61461023,61461023,Cherkaoui,Hajar,F,2000-09-10,0,2019
4,5,92839140,92839140,Kabbour,Dari,M,2000-11-05,0,2019


In [37]:
#replace the apogee column with the id column from teh students_df
final_result_df = final_result_df.merge(students_df, left_on='apogee', right_on='apogee', suffixes=('_absence', '_student'))
final_result_df.drop(columns=['code','filiere','semester','code_apo','nom','prenom','sexe','birthdate','number_of_ajournements','promo'], inplace=True)
final_result_df.drop(columns=['module','name','coeff'], inplace=True)
final_result_df.head()

,apogee,total_absences,id_absence,prof_id,id_student,promo_year
0,66141774,26.0,24,12,1,2019
1,34462736,25.0,24,12,2,2019
2,87205287,28.0,24,12,3,2019
3,61461023,31.0,24,12,4,2019
4,92839140,31.0,24,12,5,2019


In [38]:
#reanme id_absence to id_module 
final_result_df.rename(columns={'id_absence': 'module_id'}, inplace=True)

In [39]:
final_result_df.head()

,apogee,total_absences,module_id,prof_id,id_student,promo_year
0,66141774,26.0,24,12,1,2019
1,34462736,25.0,24,12,2,2019
2,87205287,28.0,24,12,3,2019
3,61461023,31.0,24,12,4,2019
4,92839140,31.0,24,12,5,2019


In [40]:
final_result_df.drop(columns=['apogee'], inplace=True)
final_result_df.head()

,total_absences,module_id,prof_id,id_student,promo_year
0,26.0,24,12,1,2019
1,25.0,24,12,2,2019
2,28.0,24,12,3,2019
3,31.0,24,12,4,2019
4,31.0,24,12,5,2019


In [41]:
#save to the final_result_df to a csv file
final_result_df.to_csv('absence_fact.csv', index=False)

In [42]:
# get marks data from the database
cursor.execute("SELECT * FROM marks")
marks = cursor.fetchall()
columns = [column[0] for column in cursor.description]
marks_df = pd.DataFrame(marks, columns=columns)
marks_df.head()


,student_apogee,module_id,mark,is_split2,mark_submodule1,mark_submodule2,year
0,66141774,8,15.46,True,14.73,16.20,2016/2017
1,34462736,8,12.75,True,12.00,13.49,2016/2017
2,87205287,8,17.09,True,20.00,14.19,2016/2017
3,61461023,8,15.40,True,17.72,13.09,2016/2017
4,92839140,8,16.07,True,19.50,12.64,2016/2017


In [43]:
students_df.head()


,id,apogee,code_apo,nom,prenom,sexe,birthdate,number_of_ajournements,promo_year
0,1,66141774,66141774,Benkirane,Fouad,M,2000-06-25,1,2019
1,2,34462736,34462736,Zayed,Dounia,F,2000-02-13,0,2019
2,3,87205287,87205287,El-Bay,Dounia,F,2000-08-31,0,2019
3,4,61461023,61461023,Cherkaoui,Hajar,F,2000-09-10,0,2019
4,5,92839140,92839140,Kabbour,Dari,M,2000-11-05,0,2019


In [44]:
#add the apoge column to the final_result_df based on the student_id column
final_result_df = final_result_df.merge(students_df, left_on='id_student', right_on='id', suffixes=('_absence', '_student'))
final_result_df.drop(columns=['id_student','id','sexe','birthdate','number_of_ajournements','promo_year_student'], inplace=True)
final_result_df.head()

,total_absences,module_id,prof_id,promo_year_absence,apogee,code_apo,nom,prenom
0,26.0,24,12,2019,66141774,66141774,Benkirane,Fouad
1,25.0,24,12,2019,34462736,34462736,Zayed,Dounia
2,28.0,24,12,2019,87205287,87205287,El-Bay,Dounia
3,31.0,24,12,2019,61461023,61461023,Cherkaoui,Hajar
4,31.0,24,12,2019,92839140,92839140,Kabbour,Dari


In [45]:
#drop the nom prenom columns
final_result_df.drop(columns=['nom','prenom'], inplace=True)
final_result_df.head()


,total_absences,module_id,prof_id,promo_year_absence,apogee,code_apo
0,26.0,24,12,2019,66141774,66141774
1,25.0,24,12,2019,34462736,34462736
2,28.0,24,12,2019,87205287,87205287
3,31.0,24,12,2019,61461023,61461023
4,31.0,24,12,2019,92839140,92839140


In [46]:
#rename promo column to promo_year
final_result_df.rename(columns={'promo_year_absence': 'promo'}, inplace=True)
final_result_df.head()

,total_absences,module_id,prof_id,promo,apogee,code_apo
0,26.0,24,12,2019,66141774,66141774
1,25.0,24,12,2019,34462736,34462736
2,28.0,24,12,2019,87205287,87205287
3,31.0,24,12,2019,61461023,61461023
4,31.0,24,12,2019,92839140,92839140


In [47]:
final_result_df.drop(columns=['code_apo'], inplace=True)

In [48]:
marks_df.head()

,student_apogee,module_id,mark,is_split2,mark_submodule1,mark_submodule2,year
0,66141774,8,15.46,True,14.73,16.20,2016/2017
1,34462736,8,12.75,True,12.00,13.49,2016/2017
2,87205287,8,17.09,True,20.00,14.19,2016/2017
3,61461023,8,15.40,True,17.72,13.09,2016/2017
4,92839140,8,16.07,True,19.50,12.64,2016/2017


In [49]:
#print the columns of the marks_df and final_result_df
print("Columns of the marks_df:")
print(marks_df.columns)
print("\nColumns of the final_result_df:")
print(final_result_df.columns)

Columns of the marks_df:
Index(['student_apogee', 'module_id', 'mark', 'is_split2', 'mark_submodule1',
       'mark_submodule2', 'year'],
      dtype='object')

Columns of the final_result_df:
Index(['total_absences', 'module_id', 'prof_id', 'promo', 'apogee'], dtype='object')


In [50]:
# Merge the two DataFrames on 'apogee' and 'module_id'
merged_df = pd.merge(final_result_df, marks_df, left_on=['apogee', 'module_id'], right_on=['student_apogee', 'module_id'])

# Drop the redundant 'student_apogee' column after the merge
merged_df.drop(columns=['student_apogee'], inplace=True)

# Print the merged DataFrame
print(merged_df.head())

   total_absences  module_id  prof_id  promo    apogee   mark  is_split2  \
0            26.0         24       12   2019  66141774  12.00      False   
1            25.0         24       12   2019  34462736  12.00      False   
2            28.0         24       12   2019  87205287  14.52      False   
3            31.0         24       12   2019  61461023  19.49      False   
4            31.0         24       12   2019  92839140  12.84      False   

   mark_submodule1  mark_submodule2       year  
0              NaN              NaN  2017/2018  
1              NaN              NaN  2017/2018  
2              NaN              NaN  2017/2018  
3              NaN              NaN  2017/2018  
4              NaN              NaN  2017/2018  


In [51]:
merged_df.drop(columns=['is_split2','mark_submodule1','mark_submodule2'], inplace=True)


In [52]:
merged_df.head()

,total_absences,module_id,prof_id,promo,apogee,mark,year
0,26.0,24,12,2019,66141774,12.00,2017/2018
1,25.0,24,12,2019,34462736,12.00,2017/2018
2,28.0,24,12,2019,87205287,14.52,2017/2018
3,31.0,24,12,2019,61461023,19.49,2017/2018
4,31.0,24,12,2019,92839140,12.84,2017/2018


In [53]:
#save the merged_df to a csv file
merged_df.to_csv('fact_student_performance.csv', index=False)

In [54]:
#load data to the DATAWAREHOUSE
#connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()


In [55]:
#load the data to the student dimension table
students_df.head()


,id,apogee,code_apo,nom,prenom,sexe,birthdate,number_of_ajournements,promo_year
0,1,66141774,66141774,Benkirane,Fouad,M,2000-06-25,1,2019
1,2,34462736,34462736,Zayed,Dounia,F,2000-02-13,0,2019
2,3,87205287,87205287,El-Bay,Dounia,F,2000-08-31,0,2019
3,4,61461023,61461023,Cherkaoui,Hajar,F,2000-09-10,0,2019
4,5,92839140,92839140,Kabbour,Dari,M,2000-11-05,0,2019


In [56]:
students_df.drop(columns=['code_apo'], inplace=True)

In [57]:
#create the student dimension table in the database and load the data
conn = psycopg2.connect(
  host="localhost",
  database="BI_DW",
  user="postgres",
  password="root"
)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_student (
    student_id SERIAL PRIMARY KEY,
    apogee VARCHAR(50) NOT NULL,
    nom VARCHAR(50) NOT NULL,
    prenom VARCHAR(50) NOT NULL,
    promo VARCHAR(50) NOT NULL,
    sexe CHAR(1) NOT NULL,
    birthdate DATE NOT NULL,
    number_of_ajournements INTEGER NOT NULL
);
""")
conn.commit()


In [58]:
# Ensure the DataFrame columns have the correct data types
students_df['apogee'] = students_df['apogee'].astype(int)
students_df['nom'] = students_df['nom'].astype(str)
students_df['prenom'] = students_df['prenom'].astype(str)
students_df['sexe'] = students_df['sexe'].astype(str)
students_df['birthdate'] = pd.to_datetime(students_df['birthdate'], format='%Y-%m-%d')
students_df['number_of_ajournements'] = students_df['number_of_ajournements'].astype(int)
students_df['promo_year'] = students_df['promo_year'].astype(int)
#delete duplicates in the students_df
students_df.drop_duplicates(subset=['apogee'], inplace=True)
# Verify the conversion

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()

# # Insert the data into the dim_student table
# for index, row in students_df.iterrows():
#     cursor.execute("""
#     INSERT INTO dim_student (apogee, nom, prenom, promo, sexe, birthdate, number_of_ajournements)
#     VALUES (%s, %s, %s, %s, %s, %s, %s)
#     """, (row['apogee'], row['nom'], row['prenom'], row['promo_year'], row['sexe'], row['birthdate'], row['number_of_ajournements']))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [59]:
#modules data load to the database
modules_df.head()

,id,code,name,filiere,coeff,prof_id,semester
0,1,TC1,Langues et Communication I,GI,0.5,17,S1
1,2,TC1,Langues et Communication I,GI,0.5,19,S1
2,3,TC2,Management I,GI,1.0,21,S1
3,4,TC3,Probabilités Statistiques et Calcul Stochastique,GI,1.0,22,S1
4,5,TC4,Réseaux Informatiques I,GI,1.0,8,S1


In [60]:
print(module_df.dtypes)

id                int64
code             object
name             object
filiere          object
coeff           float64
profID            int64
semester         object
absence_bias      int64
median_mark     float64
dtype: object


In [61]:
import pandas as pd

# Ensure the DataFrame columns have the correct data types
modules_df['id'] = modules_df['id'].astype(int)
modules_df['code'] = modules_df['code'].astype(str)
modules_df['name'] = modules_df['name'].astype(str)
modules_df['filiere'] = modules_df['filiere'].astype(str)
modules_df['coeff'] = modules_df['coeff'].astype(float)
modules_df['prof_id'] = modules_df['prof_id'].astype(int)
modules_df['semester'] = modules_df['semester'].astype(str)
module_df.drop(columns=['absence_bias','median_mark'], inplace=True)
# Verify the conversion
print(modules_df.dtypes)

id            int64
code         object
name         object
filiere      object
coeff       float64
prof_id       int64
semester     object
dtype: object


In [62]:
#import pandas as pd
import psycopg2

# Ensure the DataFrame columns have the correct data types
modules_df['id'] = modules_df['id'].astype(int)
modules_df['code'] = modules_df['code'].astype(str)
modules_df['name'] = modules_df['name'].astype(str)
modules_df['filiere'] = modules_df['filiere'].astype(str)
modules_df['coeff'] = modules_df['coeff'].astype(float)
modules_df['prof_id'] = modules_df['prof_id'].astype(int)
modules_df['semester'] = modules_df['semester'].astype(str)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()

# Create the dim_module table
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_module (
    module_id SERIAL PRIMARY KEY,
    code VARCHAR(50) NOT NULL,
    name VARCHAR(100) NOT NULL,
    filiere VARCHAR(100) NOT NULL,
    coeff NUMERIC(5,2) NOT NULL,
    semester VARCHAR(50) NOT NULL,
    prof_id INTEGER NOT NULL
);
""")

# Insert the data into the dim_module table
# for index, row in modules_df.iterrows():
#     cursor.execute("""
#     INSERT INTO dim_module (code, name, filiere, coeff, semester)
#     VALUES (%s, %s, %s, %s, %s)
#     """, (row['code'], row['name'], row['filiere'], row['coeff'], row['semester']))


# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [63]:
prof_df.head()

,id,name,departement,email
0,1,Anouar ABTOY,GI,anouar@uae.ac.ma
1,2,Mohammed AL ACHHAB,GI,mohammed@uae.ac.ma
2,3,Zineb BESRI,GI,zineb@uae.ac.ma
3,4,Mohamed Yassin CHKOURI,GI,mohamedyassin@uae.ac.ma
4,5,Mohamed CHRAYAH,GI,mohamed@uae.ac.ma


In [64]:
print(prof_df.dtypes)

id              int64
name           object
departement    object
email          object
dtype: object


In [65]:
import pandas as pd
import psycopg2

# Ensure the DataFrame columns have the correct data types
prof_df['id'] = prof_df['id'].astype(int)
prof_df['name'] = prof_df['name'].astype(str)
prof_df['departement'] = prof_df['departement'].astype(str)
prof_df['email'] = prof_df['email'].astype(str)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()

# Create the dim_professor table
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_professor (
    prof_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    departement VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL
);
""")

# Insert the data into the dim_professor table
# for index, row in prof_df.iterrows():
#     cursor.execute("""
#     INSERT INTO dim_professor (name, departement, email)
#     VALUES (%s, %s, %s)
#     """, (row['name'], row['departement'], row['email']))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [66]:
import pandas as pd
import psycopg2

# Get unique values in the year column of the marks_df
unique_years = marks_df['year'].unique()

# Get unique values in the semester column of the module_df
unique_semesters = module_df['semester'].unique()

# Create a DataFrame for the time dimension table
time_df = pd.DataFrame([(year, semester) for year in unique_years for semester in unique_semesters], columns=['year', 'semester'])

# Convert the year column to string type
time_df['year'] = time_df['year'].astype(str)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()

# Create the dim_time table
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_time (
    time_id SERIAL PRIMARY KEY,
    year VARCHAR(50) NOT NULL,
    semester VARCHAR(50) NOT NULL
);
""")

# Insert the data into the dim_time table
# for index, row in time_df.iterrows():
#     cursor.execute("""
#     INSERT INTO dim_time (year, semester)
#     VALUES (%s, %s)
#     """, (row['year'], row['semester']))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [67]:
final_result_df.head()

,total_absences,module_id,prof_id,promo,apogee
0,26.0,24,12,2019,66141774
1,25.0,24,12,2019,34462736
2,28.0,24,12,2019,87205287
3,31.0,24,12,2019,61461023
4,31.0,24,12,2019,92839140


In [68]:
print(final_result_df.dtypes)

total_absences    float64
module_id           int64
prof_id             int64
promo               int64
apogee              int64
dtype: object


In [69]:
import pandas as pd
import psycopg2
import numpy as np

# Ensure the DataFrame columns have the correct data types
final_result_df['total_absences'] = final_result_df['total_absences'].astype(float)
final_result_df['module_id'] = final_result_df['module_id'].astype(int)
final_result_df['prof_id'] = final_result_df['prof_id'].astype(int)
final_result_df['promo'] = final_result_df['promo'].astype(int)
final_result_df['apogee'] = final_result_df['apogee'].astype(int)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()

# Create the fact_student_performance table
cursor.execute("""
CREATE TABLE IF NOT EXISTS fact_student_performance (
    fact_id SERIAL PRIMARY KEY,
    total_absences FLOAT NOT NULL,
    module_id INTEGER NOT NULL,
    prof_id INTEGER NOT NULL,
    promo INTEGER NOT NULL,
    apogee INTEGER NOT NULL
);
""")

# Insert the data into the fact_student_performance table
# for index, row in final_result_df.iterrows():
#     cursor.execute("""
#     INSERT INTO fact_student_performance (total_absences, module_id, prof_id, promo, apogee)
#     VALUES (%s, %s, %s, %s, %s)
#     """, (float(row['total_absences']), int(row['module_id']), int(row['prof_id']), int(row['promo']), int(row['apogee'])))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [70]:
#time df 
time_df


,year,semester
0,2016/2017,S1
1,2016/2017,S2
2,2016/2017,S3
3,2016/2017,S4
4,2016/2017,S5
5,2017/2018,S1
6,2017/2018,S2
7,2017/2018,S3
8,2017/2018,S4
9,2017/2018,S5


In [71]:
#fact table column promo and module 
final_result_df[['promo', 'module_id']]

,promo,module_id
0,2019,24
1,2019,24
2,2019,24
3,2019,24
4,2019,24
...,...,...
17915,2024,30
17916,2024,29
17917,2024,30
17918,2024,29


In [72]:
#module_df semesters and id columns
module_df[['semester', 'id']]

,semester,id
0,S1,1
1,S1,2
2,S1,3
3,S1,4
4,S1,5
5,S1,6
6,S1,7
7,S2,8
8,S2,9
9,S2,10


In [73]:
#merge final_result_df[['promo', 'module_id']] with module_df[['semester', 'id']] on the module_id column
temp_df = final_result_df[['promo', 'module_id']].merge(module_df[['semester', 'id']], left_on='module_id', right_on='id', suffixes=('_fact', '_module'))
#drop the id column
temp_df.drop(columns=['id'], inplace=True)

In [74]:
print(temp_df)

       promo  module_id semester
0       2019         24       S4
1       2019         24       S4
2       2019         24       S4
3       2019         24       S4
4       2019         24       S4
...      ...        ...      ...
17915   2024         30       S5
17916   2024         29       S5
17917   2024         30       S5
17918   2024         29       S5
17919   2024         30       S5

[17920 rows x 3 columns]


In [75]:
print(time_df)

         year semester
0   2016/2017       S1
1   2016/2017       S2
2   2016/2017       S3
3   2016/2017       S4
4   2016/2017       S5
5   2017/2018       S1
6   2017/2018       S2
7   2017/2018       S3
8   2017/2018       S4
9   2017/2018       S5
10  2018/2019       S1
11  2018/2019       S2
12  2018/2019       S3
13  2018/2019       S4
14  2018/2019       S5
15  2019/2020       S1
16  2019/2020       S2
17  2019/2020       S3
18  2019/2020       S4
19  2019/2020       S5
20  2020/2021       S1
21  2020/2021       S2
22  2020/2021       S3
23  2020/2021       S4
24  2020/2021       S5
25  2021/2022       S1
26  2021/2022       S2
27  2021/2022       S3
28  2021/2022       S4
29  2021/2022       S5
30  2022/2023       S1
31  2022/2023       S2
32  2022/2023       S3
33  2022/2023       S4
34  2022/2023       S5
35  2023/2024       S1
36  2023/2024       S2
37  2023/2024       S3
38  2023/2024       S4
39  2023/2024       S5


In [76]:
#for promo =2019 and semster S1 the time id is 0 (year 2016/2017 and semester S1) , for promo =2019 and semster S2 the time id is 1 (year 2016/2017 and semester S2) and so on
time_df['time_id'] = time_df.index
time_df


,year,semester,time_id
0,2016/2017,S1,0
1,2016/2017,S2,1
2,2016/2017,S3,2
3,2016/2017,S4,3
4,2016/2017,S5,4
5,2017/2018,S1,5
6,2017/2018,S2,6
7,2017/2018,S3,7
8,2017/2018,S4,8
9,2017/2018,S5,9


In [77]:
def map_to_time_id(row, time_df):
  # Calculate base year (2016 for promo 2019)
  base_year = row['promo'] - 3

  # Convert semester to numeric (S1=0, S2=1, etc.)
  semester_num = int(row['semester'][1]) 

  # Find matching row in time_df
  mask = (time_df['year'] == f"{base_year}/{base_year+1}") & (time_df['semester'] == row['semester'])
  time_id = time_df.loc[mask, 'time_id'].iloc[0]

  return time_id

# Apply mapping
temp_df['time_id'] = temp_df.apply(lambda row: map_to_time_id(row, time_df), axis=1)

# Drop original columns
tmp_df = temp_df.drop(['promo', 'semester'], axis=1)

In [78]:
tmp_df.head()

,module_id,time_id
0,24,3
1,24,3
2,24,3
3,24,3
4,24,3


In [79]:
final_result_df.head()

,total_absences,module_id,prof_id,promo,apogee
0,26.0,24,12,2019,66141774
1,25.0,24,12,2019,34462736
2,28.0,24,12,2019,87205287
3,31.0,24,12,2019,61461023
4,31.0,24,12,2019,92839140


In [80]:
final_result_df=final_result_df.merge(module_df[['semester', 'id']], left_on='module_id', right_on='id', suffixes=('_fact', '_module'))
# final_result_df.drop(columns=['id'], inplace=True)
# final_result_df=final_result_df.apply(lambda row : map_to_time_id(row, time_df), axis=1)


In [81]:
final_result_df.head()

,total_absences,module_id,prof_id,promo,apogee,semester,id
0,26.0,24,12,2019,66141774,S4,24
1,25.0,24,12,2019,34462736,S4,24
2,28.0,24,12,2019,87205287,S4,24
3,31.0,24,12,2019,61461023,S4,24
4,31.0,24,12,2019,92839140,S4,24


In [82]:
final_result_df.drop(columns=['id'], inplace=True)


In [83]:
final_result_df['time_id']=final_result_df.apply(lambda row : map_to_time_id(row, time_df), axis=1)

In [84]:
final_result_df

,total_absences,module_id,prof_id,promo,apogee,semester,time_id
0,26.0,24,12,2019,66141774,S4,3
1,25.0,24,12,2019,34462736,S4,3
2,28.0,24,12,2019,87205287,S4,3
3,31.0,24,12,2019,61461023,S4,3
4,31.0,24,12,2019,92839140,S4,3
...,...,...,...,...,...,...,...
17915,31.0,30,4,2024,70364441,S5,29
17916,22.0,29,3,2024,16433245,S5,29
17917,22.0,30,4,2024,16433245,S5,29
17918,29.0,29,3,2024,99931124,S5,29


In [85]:
#get the data from db 
conn = psycopg2.connect(
    host="localhost",
    database="BI_DW",
    user="postgres",
    password="root"
)
cursor = conn.cursor()
#get the module dim table
cursor.execute("SELECT * FROM dim_module")
dim_module = cursor.fetchall()
columns = [column[0] for column in cursor.description]
dim_module_df = pd.DataFrame(dim_module, columns=columns)
dim_module_df.head()


,module_id,code,name,filiere,coeff,semester
0,1,TC1,Langues et Communication I,GI,0.50,S1
1,2,GISIAD56,Langues et Communication III,GI,1.00,S5
2,3,TC1,Langues et Communication I,GI,0.50,S1
3,4,GINF35,Langues et Communication II,GI,0.33,S3
4,5,TC2,Management I,GI,1.00,S1


In [86]:
#get the time dim table
cursor.execute("SELECT * FROM dim_time")
dim_time = cursor.fetchall()
columns = [column[0] for column in cursor.description]
dim_time_df = pd.DataFrame(dim_time, columns=columns)
dim_time_df.head()

,time_id,year,semester
0,1,2016/2017,S1
1,2,2016/2017,S2
2,3,2017/2018,S1
3,4,2017/2018,S2
4,5,2017/2018,S3


In [87]:
#get the fact table
cursor.execute("SELECT * FROM fact_student_performance")
fact_student_performance = cursor.fetchall()
columns = [column[0] for column in cursor.description]
fact_student_performance_df = pd.DataFrame(fact_student_performance, columns=columns)
fact_student_performance_df.head()


,fact_id,total_absences,module_id,prof_id,apogee,time_id
0,1,26.0,24,12,66141774,6
1,2,25.0,24,12,34462736,6
2,3,28.0,24,12,87205287,6
3,4,31.0,24,12,61461023,6
4,5,31.0,24,12,92839140,6


In [88]:
data = [
  ('2016/2017', 'S1'), ('2016/2017', 'S2'),
  ('2017/2018', 'S1'), ('2017/2018', 'S2'), ('2017/2018', 'S3'), ('2017/2018', 'S4'),
  ('2018/2019', 'S1'), ('2018/2019', 'S2'), ('2018/2019', 'S3'), ('2018/2019', 'S4'), ('2018/2019', 'S5'),
  ('2019/2020', 'S1'), ('2019/2020', 'S2'), ('2019/2020', 'S3'), ('2019/2020', 'S4'), ('2019/2020', 'S5'),
  ('2020/2021', 'S1'), ('2020/2021', 'S2'), ('2020/2021', 'S3'), ('2020/2021', 'S4'), ('2020/2021', 'S5'),
  ('2021/2022', 'S1'), ('2021/2022', 'S2'), ('2021/2022', 'S3'), ('2021/2022', 'S4'), ('2021/2022', 'S5'),
  ('2022/2023', 'S1'), ('2022/2023', 'S2'), ('2022/2023', 'S3'), ('2022/2023', 'S4'), ('2022/2023', 'S5'),
  ('2023/2024', 'S1'), ('2023/2024', 'S2'), ('2023/2024', 'S3'), ('2023/2024', 'S4'), ('2023/2024', 'S5')
]

In [89]:
time_df = pd.DataFrame(data, columns=['year', 'semester'])


In [90]:
time_df

,year,semester
0,2016/2017,S1
1,2016/2017,S2
2,2017/2018,S1
3,2017/2018,S2
4,2017/2018,S3
5,2017/2018,S4
6,2018/2019,S1
7,2018/2019,S2
8,2018/2019,S3
9,2018/2019,S4


In [91]:
#add index to the time_df
time_df['time_id'] = time_df.index+1
time_df

,year,semester,time_id
0,2016/2017,S1,1
1,2016/2017,S2,2
2,2017/2018,S1,3
3,2017/2018,S2,4
4,2017/2018,S3,5
5,2017/2018,S4,6
6,2018/2019,S1,7
7,2018/2019,S2,8
8,2018/2019,S3,9
9,2018/2019,S4,10


In [92]:
import pandas as pd

# Function to map promo and semester to time_id
def map_to_time_id(row, time_df):
    # Calculate base year (2016 for promo 2019)
    base_year = row['promo'] - 3

    # Find matching row in time_df
    mask = (time_df['year'] == f"{base_year}/{base_year+1}") & (time_df['semester'] == row['semester'])
    if not time_df.loc[mask].empty:
        time_id = time_df.loc[mask, 'time_id'].iloc[0]
    else:
        time_id = None  # or handle the case where no match is found
    return time_id

# Apply the mapping function to the DataFrame
final_result_df['time_id'] = final_result_df.apply(lambda row: map_to_time_id(row, time_df), axis=1)

In [93]:
final_result_df


,total_absences,module_id,prof_id,promo,apogee,semester,time_id
0,26.0,24,12,2019,66141774,S4,NaN
1,25.0,24,12,2019,34462736,S4,NaN
2,28.0,24,12,2019,87205287,S4,NaN
3,31.0,24,12,2019,61461023,S4,NaN
4,31.0,24,12,2019,92839140,S4,NaN
...,...,...,...,...,...,...,...
17915,31.0,30,4,2024,70364441,S5,26.0
17916,22.0,29,3,2024,16433245,S5,26.0
17917,22.0,30,4,2024,16433245,S5,26.0
17918,29.0,29,3,2024,99931124,S5,26.0


In [94]:
print(fact_student_performance_df)

       fact_id  total_absences  module_id  prof_id    apogee  time_id
0            1            26.0         24       12  66141774        6
1            2            25.0         24       12  34462736        6
2            3            28.0         24       12  87205287        6
3            4            31.0         24       12  61461023        6
4            5            31.0         24       12  92839140        6
...        ...             ...        ...      ...       ...      ...
13155    13156            31.0          2       19  70364441       22
13156    13157            22.0          2       19  16433245       22
13157    13158            22.0          2       19  16433245       22
13158    13159            29.0          2       19  99931124       22
13159    13160            29.0          2       19  99931124       22

[13160 rows x 6 columns]


In [95]:
print(dim_module_df)

    module_id      code                                               name  \
0           1       TC1                         Langues et Communication I   
1           2  GISIAD56                       Langues et Communication III   
2           3       TC1                         Langues et Communication I   
3           4    GINF35                        Langues et Communication II   
4           5       TC2                                       Management I   
5           6       TC3   Probabilités Statistiques et Calcul Stochastique   
6           7       TC4                            Réseaux Informatiques I   
7           8       TC5  Systèmes d’Information et Bases de Données Rel...   
8           9       TC6   Théories des Graphes et Recherche Opérationnelle   
9          10    GINF21                             Électronique Numérique   
10         11    GINF22                                Programmation Web I   
11         12    GINF23               Programmation C Avancé et 

In [96]:
last_try=pd.merge(fact_student_performance_df, dim_module_df[['module_id', 'semester']], on='module_id')

In [97]:
last_try

,fact_id,total_absences,module_id,prof_id,apogee,time_id,semester
0,1,26.0,24,12,66141774,6,S3
1,2,25.0,24,12,34462736,6,S3
2,3,28.0,24,12,87205287,6,S3
3,4,31.0,24,12,61461023,6,S3
4,5,31.0,24,12,92839140,6,S3
...,...,...,...,...,...,...,...
13155,13156,31.0,2,19,70364441,22,S5
13156,13157,22.0,2,19,16433245,22,S5
13157,13158,22.0,2,19,16433245,22,S5
13158,13159,29.0,2,19,99931124,22,S5


In [98]:
dim_time_df
    

,time_id,year,semester
0,1,2016/2017,S1
1,2,2016/2017,S2
2,3,2017/2018,S1
3,4,2017/2018,S2
4,5,2017/2018,S3
5,6,2017/2018,S4
6,7,2018/2019,S1
7,8,2018/2019,S2
8,9,2018/2019,S3
9,10,2018/2019,S4


In [99]:
#using the promo and semesetre as keys to get the time_id create a map 
time_map = {}
for index, row in time_df.iterrows():
    time_map[(row['year'], row['semester'])] = row['time_id']
    

In [100]:
time_map

{('2016/2017', 'S1'): 1,
 ('2016/2017', 'S2'): 2,
 ('2017/2018', 'S1'): 3,
 ('2017/2018', 'S2'): 4,
 ('2017/2018', 'S3'): 5,
 ('2017/2018', 'S4'): 6,
 ('2018/2019', 'S1'): 7,
 ('2018/2019', 'S2'): 8,
 ('2018/2019', 'S3'): 9,
 ('2018/2019', 'S4'): 10,
 ('2018/2019', 'S5'): 11,
 ('2019/2020', 'S1'): 12,
 ('2019/2020', 'S2'): 13,
 ('2019/2020', 'S3'): 14,
 ('2019/2020', 'S4'): 15,
 ('2019/2020', 'S5'): 16,
 ('2020/2021', 'S1'): 17,
 ('2020/2021', 'S2'): 18,
 ('2020/2021', 'S3'): 19,
 ('2020/2021', 'S4'): 20,
 ('2020/2021', 'S5'): 21,
 ('2021/2022', 'S1'): 22,
 ('2021/2022', 'S2'): 23,
 ('2021/2022', 'S3'): 24,
 ('2021/2022', 'S4'): 25,
 ('2021/2022', 'S5'): 26,
 ('2022/2023', 'S1'): 27,
 ('2022/2023', 'S2'): 28,
 ('2022/2023', 'S3'): 29,
 ('2022/2023', 'S4'): 30,
 ('2022/2023', 'S5'): 31,
 ('2023/2024', 'S1'): 32,
 ('2023/2024', 'S2'): 33,
 ('2023/2024', 'S3'): 34,
 ('2023/2024', 'S4'): 35,
 ('2023/2024', 'S5'): 36}

In [101]:
# Function to map promo and semester to time_id
def map_to_time_id(row, time_map):
    # Calculate base year (2016 for promo 2019)
    base_year = row['promo'] - 3
    to_year = base_year + 1
    if row['semester'] in ['S3', 'S4']:
        base_year += 1
        to_year += 1
    elif row['semester'] == 'S5':
        base_year += 2
        to_year += 2
    key = (f"{base_year}/{to_year}", row['semester'])
    time_id = time_map.get(key)
    return time_id
#test the function
map_to_time_id({'promo': 2021, 'semester': 'S3'}, time_map)

14

In [102]:
# Apply the mapping function to the DataFrame
last_try['time_id'] = last_try.apply(lambda row: map_to_time_id(row, time_map), axis=1)

KeyError: 'promo'

In [ ]:
last_try

In [103]:
#add marks to the fact table 
marks_df

,student_apogee,module_id,mark,is_split2,mark_submodule1,mark_submodule2,year
0,66141774,8,15.46,True,14.73,16.20,2016/2017
1,34462736,8,12.75,True,12.00,13.49,2016/2017
2,87205287,8,17.09,True,20.00,14.19,2016/2017
3,61461023,8,15.40,True,17.72,13.09,2016/2017
4,92839140,8,16.07,True,19.50,12.64,2016/2017
...,...,...,...,...,...,...,...
13015,48673398,7,13.78,False,NaN,NaN,2021/2022
13016,79453784,7,13.67,False,NaN,NaN,2021/2022
13017,70364441,7,17.53,False,NaN,NaN,2021/2022
13018,16433245,7,20.00,False,NaN,NaN,2021/2022


In [104]:
#using the apogee and module_id as keys to get the mark create a column id the last_try df 
marks_df.columns

Index(['student_apogee', 'module_id', 'mark', 'is_split2', 'mark_submodule1',
       'mark_submodule2', 'year'],
      dtype='object')

In [105]:
last_try.columns


Index(['fact_id', 'total_absences', 'module_id', 'prof_id', 'apogee',
       'time_id', 'semester'],
      dtype='object')

In [107]:
# Ensure both 'apogee' columns are of the same type
last_try['apogee'] = last_try['apogee'].astype(str)
marks_df['student_apogee'] = marks_df['student_apogee'].astype(str)

# Merge last_try with marks_df on 'apogee' and 'module_id'
merged_df = pd.merge(last_try, marks_df[['student_apogee', 'module_id', 'mark']], 
                     left_on=['apogee', 'module_id'], 
                     right_on=['student_apogee', 'module_id'], 
                     how='left')

# Drop the redundant 'student_apogee' column after the merge
merged_df.drop(columns=['student_apogee'], inplace=True)

# Print the merged DataFrame to verify the result
print(merged_df.head())

# Update last_try with the new merged DataFrame
last_try = merged_df

   fact_id  total_absences  module_id  prof_id    apogee  time_id semester  \
0        1            26.0         24       12  66141774        6       S3   
1        2            25.0         24       12  34462736        6       S3   
2        3            28.0         24       12  87205287        6       S3   
3        4            31.0         24       12  61461023        6       S3   
4        5            31.0         24       12  92839140        6       S3   

    mark  
0  12.00  
1  12.00  
2  14.52  
3  19.49  
4  12.84  


In [108]:
last_try

,fact_id,total_absences,module_id,prof_id,apogee,time_id,semester,mark
0,1,26.0,24,12,66141774,6,S3,12.00
1,2,25.0,24,12,34462736,6,S3,12.00
2,3,28.0,24,12,87205287,6,S3,14.52
3,4,31.0,24,12,61461023,6,S3,19.49
4,5,31.0,24,12,92839140,6,S3,12.84
...,...,...,...,...,...,...,...,...
13155,13156,31.0,2,19,70364441,22,S5,15.65
13156,13157,22.0,2,19,16433245,22,S5,12.77
13157,13158,22.0,2,19,16433245,22,S5,12.77
13158,13159,29.0,2,19,99931124,22,S5,13.62


In [111]:
#drop column semester 
last_try.drop(columns=['semester'], inplace=True)
last_try

,fact_id,total_absences,module_id,prof_id,apogee,time_id,mark
0,1,26.0,24,12,66141774,6,12.00
1,2,25.0,24,12,34462736,6,12.00
2,3,28.0,24,12,87205287,6,14.52
3,4,31.0,24,12,61461023,6,19.49
4,5,31.0,24,12,92839140,6,12.84
...,...,...,...,...,...,...,...
13155,13156,31.0,2,19,70364441,22,15.65
13156,13157,22.0,2,19,16433245,22,12.77
13157,13158,22.0,2,19,16433245,22,12.77
13158,13159,29.0,2,19,99931124,22,13.62


In [112]:
last_try.columns

Index(['fact_id', 'total_absences', 'module_id', 'prof_id', 'apogee',
       'time_id', 'mark'],
      dtype='object')

In [113]:
#data type of the columns in the last_try df
last_try.dtypes

fact_id             int64
total_absences    float64
module_id           int64
prof_id             int64
apogee             object
time_id             int64
mark              float64
dtype: object